<a href="https://colab.research.google.com/github/benedettoscala/ifttt-code-generator/blob/main/fine_tuning_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install transformers==4.36.2
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece
!pip install -U bitsandbytes

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import torch
import pandas as pd
from datasets import Dataset


In [3]:
from google.colab import userdata
secret_hf = userdata.get('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `prova` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `prova`


# Preprocessing dei dati

In [4]:
!git clone https://github.com/benedettoscala/ifttt-code-generator
%cd ifttt-code-generator/
!git pull

Cloning into 'ifttt-code-generator'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 23 (delta 5), reused 14 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 25.20 KiB | 460.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.
Filtering content: 100% (4/4), 171.23 MiB | 32.45 MiB/s, done.
/content/ifttt-code-generator
Already up to date.


In [5]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)

In [6]:
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import BitsAndBytesConfig

#Configurazione modello base e quantizzazione
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
)

#Caricamento del modello e tokenizer
print("Caricamento del modello base...")
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

print("Caricamento del tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Prepara il modello per k-bit training (disabilita gradienti su pesi int4 ecc.)
model = prepare_model_for_kbit_training(model)

# 3. Configurazione LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"]
    # i moduli dei transformer li ho controllati su hugging face, sono questi (godo)
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()


Caricamento del modello base...


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

trainable params: 23,068,672 || all params: 7,264,800,768 || trainable%: 0.3175


In [10]:
# Caricamento e pulizia del dataset
csv_path = "datasets/cleaned_and_combined.csv"
df = pd.read_csv(csv_path)

df.dropna(subset=["cleaned_description", "filter_code"], inplace=True)

df.drop_duplicates(subset=["cleaned_description","filter_code"], inplace=True)

#Suddivisione train   e val
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset  = Dataset.from_pandas(eval_df)

In [11]:
def tokenize_function(examples):
    return tokenizer(
        examples["cleaned_description"],
        text_target=examples["filter_code"],
        truncation=True,
        max_length=512,
        padding="max_length"  # se vuoi padding dinamico, sostituisci con "longest"
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset  = eval_dataset.map(tokenize_function,  batched=True)


# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, pad_to_multiple_of=8)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [14]:
#Impostazioni di training
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,                     # aumentato a 3 epoche
    per_device_train_batch_size=2,          # batch per GPU
    gradient_accumulation_steps=4,          # accumula gradienti -> eff. batch = 8
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",             # scheduler "cosine"
    learning_rate=1e-4,                     # ridotto rispetto a 2e-4
    weight_decay=0.01,                      # alzato a 0.01
    max_grad_norm=1.0,                      # grad clip standard
    save_steps=100,                         # salva meno frequentemente
    logging_steps=5,                       # log ogni 10 step
    evaluation_strategy="steps",            # valuta ogni X step
    eval_steps=5,                         # ogni step fai eval
    load_best_model_at_end=True,            # carica modello migliore
    save_total_limit=3,                     # mantieni solo 3 checkpoint
    fp16=False,                             # se la tua GPU non supporta BF16, puoi abilitare fp16
    bf16=False,                             # se la tua GPU supporta BF16, puoi metterlo a True
    report_to="wandb"                        # o "tensorboard", "wandb", ...
)


def compute_metrics(eval_pred):
    return {}

#Creazione Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-836b77384c4f>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

#Salvataggio finale LoRA + quantization
trainer.save_model("./results/best_model")

print("Fine del training!")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
5,2.254900,1.932374
10,1.681000,1.433474
15,1.315600,1.243403
20,1.204800,1.109413


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Specifica il modello base e la cartella dove hai salvato il fine-tuning
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
FINETUNED_MODEL_DIR = "./results/best_model"

#  Carica il tokenizer del modello base
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token  # assicurati che il token di padding sia quello di fine sequenza

model_base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",         # mappa automaticamente su GPU/CPU
    torch_dtype=torch.float16,  # usa float16; se la tua GPU supporta bf16, puoi mettere torch.bfloat16
)

#Carica i pesi LoRA salvati (PeftModel)
model = PeftModel.from_pretrained(
    model_base,
    FINETUNED_MODEL_DIR,
    torch_dtype=torch.float16   # o bfloat16, in coerenza con quanto fatto su base_model
)

# Metti il modello in eval
model.eval()

#Funzione di generazione di codice (o di testo)
def generate_code(prompt, max_new_tokens=128):
    """
    prompt: testo di input (descrizione, istruzioni, ecc.)
    max_new_tokens: numero massimo di token di output
    """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # sposta su GPU
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,         # se vuoi campionare
            top_p=0.9,              # top-p sampling
            temperature=0.8,        # regola la "creatività"
            repetition_penalty=1.1  # leggero penalty su ripetizioni
        )
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)


test_prompt = (
    "if the current hour is 17, send a tweet"
)
generated = generate_code(test_prompt, max_new_tokens=100)
print("PROMPT:\n", test_prompt)
print("\nCODICE GENERATO:\n", generated)
